In [1]:

import pandas as pd

In [2]:
fil=pd.read_csv("/home/elias/code/camv98/SENDA_Chile/raw_data/hospitales_filt/2016_filtrado.csv")
hospitales_conservar=fil["Nombre Establecimiento"].unique()
df = pd.read_csv('raw_data/consolidado_2014.csv')
df_filtrado_hospital = df[df['Nombre Establecimiento'].isin(hospitales_conservar)]
df_filtrado_hospital = df_filtrado_hospital.drop(columns=['Unnamed: 0'])
correcciones = {
    'Ñuble': 'Ñuble', 
    'Ais�n': 'Aisén', 
    'Araucan�a Sur': 'Araucanía Sur',
    'Araucan�a Norte': 'Araucanía Norte', 
    'Del Reloncav�': 'Del Reloncaví', 
    '�uble': 'Ñuble', 
    'Chilo�': 'Chiloé',
    'Concepci�n': 'Concepción',
    'Los R�os': 'Los Ríos', 
    'Valdivia': 'Valdivia', 
    'Metropolitano Sur Oriente': 'Metropolitano Sur', 
    'Valparaíso San Antonio': 'Metropolitano Central'
}
correccion_columnas = {
    'C�d. SS/SEREMI': 'Cod. SS/SEREMI',
    'Nombre SS/SEREMI': 'Nombre SS/SEREMI',
    'C�d. Estab.': 'Cod. Estab.',
    'Nombre Establecimiento': 'Nombre Establecimiento',
    'C�d. Nivel Cuidado': 'Cod. Nivel Cuidado',
    'Nombre Nivel Cuidadoestan': 'Nombre Nivel Cuidado'  # Corrige el error de concatenación
}
def corregir_nombres_columnas(df):
    # Renombrar columnas
    df = df.rename(columns=correccion_columnas)
    
    # Verificar y corregir posibles problemas adicionales
    if 'Nombre Nivel Cuidadoestan' in df.columns:
        df = df.rename(columns={'Nombre Nivel Cuidadoestan': 'Nombre Nivel Cuidado'})
    
    return df
# Función para corregir los nombres de SEREMI en df_clima
def corregir_seremi(df, columna, correcciones):
    df[columna] = df[columna].replace(correcciones)
    return df

# Corregir los SEREMIs en df_clima

# Corregir los SEREMIs en df_filtrado (suponiendo que es el DataFrame de hospitales)
df_filtrado_hospital = corregir_seremi(df_filtrado_hospital , 'Nombre SS/SEREMI', correcciones)
df_filtrado_hospital = corregir_nombres_columnas(df_filtrado_hospital)
df_filtrado_hospital.to_csv("raw_data/consolidado_2014_fil.csv")



In [3]:
df_filtrado = df_filtrado_hospital[
    df_filtrado_hospital['Nombre Nivel Cuidado'] == 'Datos Establecimiento'
].copy()

# 2. Transformar a formato largo (melt)
df_melted = df_filtrado.melt(
    id_vars=[
        'Año', 'Cod. SS/SEREMI', 'Nombre SS/SEREMI', 'Cod. Estab.', 
        'Nombre Establecimiento', 'Cod. Nivel Cuidado', 'Nombre Nivel Cuidado', 'Glosa', 'Acum'
    ],
    value_vars=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'],
    var_name='Mes',
    value_name='Valor'
)

# 3. Mapear meses a números y crear fecha
meses_map = {
    'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04', 'May': '05', 'Jun': '06',
    'Jul': '07', 'Ago': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dic': '12'
}
df_melted['Fecha'] = pd.to_datetime(
    df_melted['Año'].astype(str) + '-' + df_melted['Mes'].map(meses_map) + '-01'
)

# 4. Pivotar para convertir 'Glosa' en columnas
df_final = df_melted.pivot_table(
    index=[
        'Fecha', 'Año', 'Mes', 'Cod. SS/SEREMI', 'Nombre SS/SEREMI', 
        'Cod. Estab.', 'Nombre Establecimiento', 'Cod. Nivel Cuidado', 'Nombre Nivel Cuidado'
    ],
    columns='Glosa',
    values='Valor',
    aggfunc='first'  # Usar 'first' si hay duplicados
).reset_index()

# 5. Limpiar y ordenar
df_final.columns.name = None  # Eliminar el nombre de las columnas
df_final = df_final.sort_values(['Nombre Establecimiento', 'Fecha'])

# Mostrar resultado (primeras filas)
print(f"Registros finales: {len(df_final)}")
df_final=df_final.drop(columns={"Egresos Fallecidos","Año","Mes","Cod. SS/SEREMI","Indice Ocupacional","Cod. Estab.","Indice de Rotaci�n","Letalidad","Numero de Egresos","Promedio Cama Disponibles","Nombre Nivel Cuidado","Promedio D�as de Estada","Cod. Nivel Cuidado"})

Registros finales: 6968


In [4]:
df_final = df_final.rename(columns={"Nombre SS/SEREMI": "SEREMI"})
df_final=df_final.reset_index(drop=True)
df_final.head(10)
df_final['Fecha'] = pd.to_datetime(df_final['Fecha'])


In [5]:

df_clima = pd.read_csv('/home/elias/code/camv98/SENDA_Chile/raw_data/features/todos_los_seremi_2014.csv', sep=',')
df_list = []
df = pd.read_csv("raw_data/features/Los Ríos.csv")
df['SEREMI'] ="Los Ríos"
df_list.append(df)
df_list.append(df_clima)
df_total_clima = pd.concat(df_list)


In [6]:
df_total_clima['Fecha'] = pd.to_datetime(df_total_clima['Mes'] + '-01')
df_total_clima=df_total_clima.drop(columns={"Mes","Unnamed: 0"})
df_total_clima['Fecha'] = pd.to_datetime(df_total_clima['Fecha'])
df_total_clima

,temperature_2m_max,temperature_2m_min,precipitation_sum,winddirection_10m_dominant,SEREMI,Fecha
0,21.870968,12.419355,53.0,227.064516,Los Ríos,2014-01-01
1,21.885714,12.567857,25.1,219.000000,Los Ríos,2014-02-01
2,18.648387,10.603226,104.0,227.387097,Los Ríos,2014-03-01
3,15.670000,8.460000,98.3,236.000000,Los Ríos,2014-04-01
4,13.274194,8.409677,296.5,213.419355,Los Ríos,2014-05-01
...,...,...,...,...,...,...
3745,18.422581,7.722581,49.1,216.483871,Concepción,2024-10-01
3746,19.496667,8.806667,22.8,252.333333,Concepción,2024-11-01
3747,22.490323,10.851613,14.6,242.451613,Concepción,2024-12-01
3748,25.103226,12.732258,1.6,247.709677,Concepción,2025-01-01


In [7]:
print("SEREMIs únicos en df_final:", sorted(df_final['SEREMI'].unique()))
print("SEREMIs únicos en df_clima:", sorted(df_total_clima['SEREMI'].unique()))

SEREMIs únicos en df_final: ['Aconcagua', 'Aisén', 'Antofagasta', 'Araucanía Norte', 'Araucanía Sur', 'Arauco', 'Chiloé', 'Concepción', 'Coquimbo', 'Del Libertador B.O Higgins', 'Del Maule', 'Del Reloncaví', 'Los Ríos', 'Metropolitano Occidente', 'Metropolitano Oriente', 'Metropolitano Sur', 'Osorno', 'Talcahuano', 'Valdivia', 'Ñuble']
SEREMIs únicos en df_clima: ['Aconcagua', 'Aisén', 'Antofagasta', 'Araucanía Norte', 'Araucanía Sur', 'Arauco', 'Atacama', 'Biobío', 'Chiloé', 'Concepción', 'Coquimbo', 'Del Libertador B.O Higgins', 'Del Maule', 'Del Reloncaví', 'Iquique', 'Los Ríos', 'Magallanes', 'Metropolitano Central', 'Metropolitano Norte', 'Metropolitano Occidente', 'Metropolitano Oriente', 'Metropolitano Sur', 'Metropolitano Sur Oriente', 'Osorno', 'Talcahuano', 'Valdivia', 'Valparaíso San Antonio', 'Viña del Mar Quillota', 'Ñuble']


In [8]:

print("Columnas en df_final:", df_final.columns.tolist())
print("Columnas en df_total_clima:", df_total_clima.columns.tolist())
df_final['Fecha'] = pd.to_datetime(df_final['Fecha'])
df_total_clima['Fecha'] = pd.to_datetime(df_total_clima['Fecha'])
df_final['SEREMI'] = df_final['SEREMI'].str.replace('Región ', '')
df_total_clima['SEREMI'] = df_total_clima['SEREMI'].str.strip() 
df_completo = pd.merge(
    df_final,
    df_total_clima,
    on=['Fecha', 'SEREMI'],
    how='left',
    validate='many_to_one'  # Verifica relación
)
print("\nRegistros originales:", len(df_final))
print("Registros combinados:", len(df_completo))
print("Registros sin datos climáticos:", df_completo['temperature_2m_max'].isna().sum())

Columnas en df_final: ['Fecha', 'SEREMI', 'Nombre Establecimiento', 'Dias Cama Disponibles', 'Dias Cama Ocupados', 'Dias de Estada']
Columnas en df_total_clima: ['temperature_2m_max', 'temperature_2m_min', 'precipitation_sum', 'winddirection_10m_dominant', 'SEREMI', 'Fecha']

Registros originales: 6968
Registros combinados: 6968
Registros sin datos climáticos: 9


In [9]:
df_completo[df_completo['temperature_2m_max'].isna()]

,Fecha,SEREMI,Nombre Establecimiento,Dias Cama Disponibles,Dias Cama Ocupados,Dias de Estada,temperature_2m_max,temperature_2m_min,precipitation_sum,winddirection_10m_dominant
1833,2021-08-01,Del Maule,Hospital San Juan de Dios (Cauquenes),3659.0,1397.0,1424.0,NaN,NaN,NaN,NaN
3009,2019-02-01,Del Reloncaví,Hospital de Calbuco,952.0,584.0,559.0,NaN,NaN,NaN,NaN
3843,2021-08-01,Del Maule,Hospital de Curepto,257.0,195.0,312.0,NaN,NaN,NaN,NaN
3947,2019-02-01,Del Reloncaví,Hospital de Fresia,543.0,370.0,307.0,NaN,NaN,NaN,NaN
4081,2019-02-01,Del Reloncaví,Hospital de Frutillar,896.0,632.0,591.0,NaN,NaN,NaN,NaN
4885,2019-02-01,Del Reloncaví,Hospital de Llanquihue,420.0,149.0,267.0,NaN,NaN,NaN,NaN
5719,2021-08-01,Del Maule,Hospital de Molina,1922.0,1016.0,847.0,NaN,NaN,NaN,NaN
6359,2019-02-01,Del Reloncaví,Hospital de Puerto Montt,14392.0,12781.0,11734.0,NaN,NaN,NaN,NaN
6791,2021-08-01,Del Maule,Hospital de Teno,868.0,535.0,505.0,NaN,NaN,NaN,NaN


In [10]:
import numpy as np
def rellenar_nan_promedio_mes(df, columna):
    """Rellena los NaN de una columna con el promedio de un mes antes y después."""
    for i, row in df.iterrows():
        if pd.isna(row[columna]):
            fecha_actual = row['Fecha']
            mes_anterior = fecha_actual - pd.DateOffset(months=1)
            mes_siguiente = fecha_actual + pd.DateOffset(months=1)

            valores_mes_anterior = df.loc[df['Fecha'] == mes_anterior, columna].values
            valores_mes_siguiente = df.loc[df['Fecha'] == mes_siguiente, columna].values

            valores_validos = []
            if len(valores_mes_anterior) > 0 and not pd.isna(valores_mes_anterior[0]):
                valores_validos.append(valores_mes_anterior[0])
            if len(valores_mes_siguiente) > 0 and not pd.isna(valores_mes_siguiente[0]):
                valores_validos.append(valores_mes_siguiente[0])

            if len(valores_validos) > 0:
                df.at[i, columna] = np.mean(valores_validos)
    return df

In [11]:
columnas_a_rellenar = ['temperature_2m_max', 'temperature_2m_min', 'precipitation_sum', 'winddirection_10m_dominant']

for columna in columnas_a_rellenar:
    df_completo = rellenar_nan_promedio_mes(df_completo, columna)

df_completo

,Fecha,SEREMI,Nombre Establecimiento,Dias Cama Disponibles,Dias Cama Ocupados,Dias de Estada,temperature_2m_max,temperature_2m_min,precipitation_sum,winddirection_10m_dominant
0,2014-01-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),3722.0,2755.0,2983.0,28.490323,14.135484,0.3,248.064516
1,2014-02-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),3364.0,2494.0,2325.0,27.089286,12.617857,4.1,252.607143
2,2014-03-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),3729.0,2804.0,2629.0,26.216129,11.858065,4.8,254.935484
3,2014-04-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),3607.0,2706.0,2696.0,22.810000,8.843333,0.4,247.733333
4,2014-05-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),3723.0,2669.0,2899.0,18.145161,6.619355,19.5,274.451613
...,...,...,...,...,...,...,...,...,...,...
6963,2024-10-01,Araucanía Sur,Hospital de Villarrica,2954.0,2678.0,2617.0,12.338710,1.193548,75.7,252.354839
6964,2024-11-01,Araucanía Sur,Hospital de Villarrica,2774.0,2488.0,2451.0,12.496667,2.323333,198.3,271.766667
6965,2024-12-01,Araucanía Sur,Hospital de Villarrica,3008.0,2540.0,2311.0,16.580645,4.348387,49.6,246.774194
6966,2025-01-01,Araucanía Sur,Hospital de Villarrica,3012.0,2735.0,2917.0,20.622581,5.932258,32.9,253.258065


In [12]:
df_completo[df_completo['temperature_2m_max'].isna()]

,Fecha,SEREMI,Nombre Establecimiento,Dias Cama Disponibles,Dias Cama Ocupados,Dias de Estada,temperature_2m_max,temperature_2m_min,precipitation_sum,winddirection_10m_dominant


In [13]:
def agregar_funciones_ciclicas_tiempo(df, columna_fecha='Fecha'):
    """Agrega columnas seno y coseno para representar el tiempo de forma cíclica."""
    
    # Convertir la columna de fecha a datetime si no lo está
    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    
    # Calcular el número de días desde el inicio del dataset
    df['dias_desde_inicio'] = (df[columna_fecha] - df[columna_fecha].min()).dt.days
    
    # Calcular representaciones cíclicas del tiempo (anual y mensual)
    
    df['seno_mensual'] = np.sin(2 * np.pi * df['dias_desde_inicio'] / 30.44) # promedio de días en un mes
    df['coseno_mensual'] = np.cos(2 * np.pi * df['dias_desde_inicio'] / 30.44)
    
    return df

In [14]:
df = agregar_funciones_ciclicas_tiempo(df_completo)
df

,Fecha,SEREMI,Nombre Establecimiento,Dias Cama Disponibles,Dias Cama Ocupados,Dias de Estada,temperature_2m_max,temperature_2m_min,precipitation_sum,winddirection_10m_dominant,dias_desde_inicio,seno_mensual,coseno_mensual
0,2014-01-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),3722.0,2755.0,2983.0,28.490323,14.135484,0.3,248.064516,0,0.000000,1.000000
1,2014-02-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),3364.0,2494.0,2325.0,27.089286,12.617857,4.1,252.607143,31,0.115334,0.993327
2,2014-03-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),3729.0,2804.0,2629.0,26.216129,11.858065,4.8,254.935484,59,-0.378389,0.925647
3,2014-04-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),3607.0,2706.0,2696.0,22.810000,8.843333,0.4,247.733333,90,-0.269105,0.963111
4,2014-05-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),3723.0,2669.0,2899.0,18.145161,6.619355,19.5,274.451613,120,-0.355347,0.934734
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6963,2024-10-01,Araucanía Sur,Hospital de Villarrica,2954.0,2678.0,2617.0,12.338710,1.193548,75.7,252.354839,3926,-0.156231,0.987721
6964,2024-11-01,Araucanía Sur,Hospital de Villarrica,2774.0,2488.0,2451.0,12.496667,2.323333,198.3,271.766667,3957,-0.041271,0.999148
6965,2024-12-01,Araucanía Sur,Hospital de Villarrica,3008.0,2540.0,2311.0,16.580645,4.348387,49.6,246.774194,3987,-0.131720,0.991287
6966,2025-01-01,Araucanía Sur,Hospital de Villarrica,3012.0,2735.0,2917.0,20.622581,5.932258,32.9,253.258065,4018,-0.016512,0.999864


In [22]:
y = df["Dias Cama Disponibles"].copy() 
X = df.drop(columns=["Dias Cama Disponibles"]) 

In [23]:
X

,Fecha,SEREMI,Nombre Establecimiento,Dias Cama Ocupados,Dias de Estada,temperature_2m_max,temperature_2m_min,precipitation_sum,winddirection_10m_dominant,dias_desde_inicio,seno_mensual,coseno_mensual
0,2014-01-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),2755.0,2983.0,28.490323,14.135484,0.3,248.064516,0,0.000000,1.000000
1,2014-02-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),2494.0,2325.0,27.089286,12.617857,4.1,252.607143,31,0.115334,0.993327
2,2014-03-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),2804.0,2629.0,26.216129,11.858065,4.8,254.935484,59,-0.378389,0.925647
3,2014-04-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),2706.0,2696.0,22.810000,8.843333,0.4,247.733333,90,-0.269105,0.963111
4,2014-05-01,Metropolitano Occidente,Hospital Adalberto Steeger (Talagante),2669.0,2899.0,18.145161,6.619355,19.5,274.451613,120,-0.355347,0.934734
...,...,...,...,...,...,...,...,...,...,...,...,...
6963,2024-10-01,Araucanía Sur,Hospital de Villarrica,2678.0,2617.0,12.338710,1.193548,75.7,252.354839,3926,-0.156231,0.987721
6964,2024-11-01,Araucanía Sur,Hospital de Villarrica,2488.0,2451.0,12.496667,2.323333,198.3,271.766667,3957,-0.041271,0.999148
6965,2024-12-01,Araucanía Sur,Hospital de Villarrica,2540.0,2311.0,16.580645,4.348387,49.6,246.774194,3987,-0.131720,0.991287
6966,2025-01-01,Araucanía Sur,Hospital de Villarrica,2735.0,2917.0,20.622581,5.932258,32.9,253.258065,4018,-0.016512,0.999864


In [61]:
import pandas as pd
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 1. Función mejorada de feature engineering
def enhanced_features(X):
    # Convertir a DataFrame si es necesario
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=input_columns)  # Asegúrate de definir input_columns
    
    # Ordenar por hospital y fecha
    X = X.sort_values(["Nombre Establecimiento", "Fecha"])
    
    # Crear múltiples lags
    for lag in [1, 2, 3, 6, 12]:
        X[f'ocupados_lag_{lag}'] = X.groupby("Nombre Establecimiento")["Dias Cama Ocupados"].shift(lag)
        X[f'estada_lag_{lag}'] = X.groupby("Nombre Establecimiento")["Dias de Estada"].shift(lag)
    
    # Features adicionales
    X['ocupados_rolling_3'] = X.groupby("Nombre Establecimiento")["Dias Cama Ocupados"].transform(lambda x: x.rolling(3).mean())
    X['ocupados_diff_12'] = X.groupby("Nombre Establecimiento")["Dias Cama Ocupados"].diff(12)
    
    # Eliminar columnas originales
    return X.drop(columns=["Dias Cama Ocupados", "Dias de Estada"])

# 2. Función para nombres de features (compatible con sklearn)
def get_feature_names_out(self, input_features=None):
    # Columnas base (excluyendo las eliminadas)
    base_cols = [col for col in input_features if col not in ["Dias Cama Ocupados", "Dias de Estada"]]
    
    # Nuevas features generadas
    new_features = (
        [f'ocupados_lag_{i}' for i in [1, 2, 3, 6, 12]] +
        [f'estada_lag_{i}' for i in [1, 2, 3, 6, 12]] +
        ['ocupados_rolling_3', 'ocupados_diff_12']
    )
    return base_cols + new_features

# 3. Configuración del transformer
lag_transformer = FunctionTransformer(
    enhanced_features,
    feature_names_out=get_feature_names_out
)

# 4. Preprocessor para otras transformaciones
preprocessor = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore'), ['Nombre Establecimiento']),
    ('drop', 'drop', ['SEREMI', 'Fecha'])
], remainder='passthrough')

# 5. Pipeline completo
pipeline = Pipeline([
    ('add_features', lag_transformer),
    ('preprocess', preprocessor)
]).set_output(transform='pandas')  # Salida como DataFrame

# 6. Aplicación
try:
    X_transformed = pipeline.fit_transform(X)
    print("Transformación exitosa!")
    print("\nColumnas resultantes:", X_transformed.columns.tolist())
except Exception as e:
    print(f"Error: {str(e)}")


Error: The output generated by `func` have different column names than the ones provided by `get_feature_names_out`. Got output with columns names: ['Fecha', 'SEREMI', 'Nombre Establecimiento', 'temperature_2m_max', 'temperature_2m_min', 'precipitation_sum', 'winddirection_10m_dominant', 'dias_desde_inicio', 'seno_mensual', 'coseno_mensual', 'ocupados_lag_1', 'estada_lag_1', 'ocupados_lag_2', 'estada_lag_2', 'ocupados_lag_3', 'estada_lag_3', 'ocupados_lag_6', 'estada_lag_6', 'ocupados_lag_12', 'estada_lag_12', 'ocupados_rolling_3', 'ocupados_rolling_12', 'ocupados_diff_12'] and `get_feature_names_out` returned: ['Fecha', 'SEREMI', 'Nombre Establecimiento', 'temperature_2m_max', 'temperature_2m_min', 'precipitation_sum', 'winddirection_10m_dominant', 'dias_desde_inicio', 'seno_mensual', 'coseno_mensual', 'ocupados_lag_1', 'estada_lag_1', 'ocupados_lag_2', 'estada_lag_2', 'ocupados_lag_3', 'estada_lag_3', 'ocupados_lag_6', 'estada_lag_6', 'ocupados_lag_12', 'estada_lag_12', 'ocupados_rol

In [49]:

X_transformed_df = X_transformed_df.dropna()
y_clean = y[X_transformed_df.index]
X_transformed_df

,onehot__Nombre Establecimiento_Hospital Adalberto Steeger (Talagante),"onehot__Nombre Establecimiento_Hospital Barros Luco Trudeau (Santiago, San Miguel)",onehot__Nombre Establecimiento_Hospital Clorinda Avello (Santa Juana),onehot__Nombre Establecimiento_Hospital Comunitario de Salud Familiar Pedro Morales Campos (Yungay),onehot__Nombre Establecimiento_Hospital Comunitario de Salud Familiar de Bulnes,onehot__Nombre Establecimiento_Hospital Comunitario de Salud Familiar de El Carmen,onehot__Nombre Establecimiento_Hospital Comunitario de Salud Familiar de Quirihue,"onehot__Nombre Establecimiento_Hospital El Pino (Santiago, San Bernardo)",onehot__Nombre Establecimiento_Hospital Hanga Roa (Isla De Pascua),onehot__Nombre Establecimiento_Hospital Las Higueras (Talcahuano),...,onehot__Nombre Establecimiento_Hospital de Villarrica,remainder__temperature_2m_max,remainder__temperature_2m_min,remainder__precipitation_sum,remainder__winddirection_10m_dominant,remainder__dias_desde_inicio,remainder__seno_mensual,remainder__coseno_mensual,remainder__Dias_Cama_Ocupados_lag1,remainder__Dias_de_Estada_lag1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,27.089286,12.617857,4.1,252.607143,31.0,0.115334,0.993327,2755.0,2983.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,26.216129,11.858065,4.8,254.935484,59.0,-0.378389,0.925647,2494.0,2325.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,22.810000,8.843333,0.4,247.733333,90.0,-0.269105,0.963111,2804.0,2629.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,18.145161,6.619355,19.5,274.451613,120.0,-0.355347,0.934734,2706.0,2696.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,14.023333,2.050000,112.7,262.033333,151.0,-0.245170,0.969480,2669.0,2899.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,12.338710,1.193548,75.7,252.354839,3926.0,-0.156231,0.987721,2235.0,1974.0
6964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,12.496667,2.323333,198.3,271.766667,3957.0,-0.041271,0.999148,2678.0,2617.0
6965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,16.580645,4.348387,49.6,246.774194,3987.0,-0.131720,0.991287,2488.0,2451.0
6966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,20.622581,5.932258,32.9,253.258065,4018.0,-0.016512,0.999864,2540.0,2311.0


In [62]:
import numpy as np

# Paso 1: Obtener el ID del hospital (índice de la columna one-hot con valor 1.0)
hospital_ids = np.argmax(X_transformed_df.iloc[:, :52].values, axis=1)  # 52 columnas one-hot

# Paso 2: Añadir esta columna al DataFrame (opcional)
X_transformed_df['hospital_id'] = hospital_ids

# Paso 3: Usar GroupShuffleSplit
splitter = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(splitter.split(X_transformed_df, groups=X_transformed_df['hospital_id']))

X_train, X_test = X_transformed_df.iloc[train_idx], X_transformed_df.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [65]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Evaluar
y_pred = model.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")
print(f"R²: {model.score(X_test, y_test)}")

MAE: 1208.093926862611
R²: 0.3581146269169314


In [69]:
from xgboost import XGBRegressor
xgb_params = {
    'n_estimators': 500,
    'learning_rate': 0.05,
    'max_depth': 8,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0.1,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': 42,
    'n_jobs': -1
}
model = XGBRegressor(**xgb_params)
model.fit(X_train, y_train)

# Evaluar
y_pred = model.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")

MAE: 1187.108360556487


In [83]:
print("Columnas disponibles:", X.columns.tolist())

Columnas disponibles: ['Fecha', 'SEREMI', 'Nombre Establecimiento', 'Dias Cama Ocupados', 'Dias de Estada', 'temperature_2m_max', 'temperature_2m_min', 'precipitation_sum', 'winddirection_10m_dominant', 'dias_desde_inicio', 'seno_mensual', 'coseno_mensual', 'ocupados_lag_1', 'estada_lag_1', 'ocupados_lag_2', 'estada_lag_2', 'ocupados_lag_3', 'estada_lag_3', 'ocupados_lag_6', 'estada_lag_6', 'ocupados_lag_12', 'estada_lag_12', 'ocupados_rolling_3', 'ocupados_rolling_12', 'ocupados_diff_12']


In [91]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

# 1. Definir columnas base que SÍ existen en todos los hospitales
COLUMNAS_BASE = [
    'Nombre Establecimiento',
    'temperature_2m_max',
    'temperature_2m_min',
    'dias_desde_inicio'
]

# 2. Columnas adicionales (verificar existencia)
COLUMNAS_EXTRA = [
    'precipitation_sum',
    'winddirection_10m_dominant',
    'seno_mensual',
    'coseno_mensual',
    'ocupados_lag_1',
    'ocupados_lag_3',
    'ocupados_rolling_3'
]

# 3. Función para verificar columnas existentes
def get_valid_columns(df, base_cols, extra_cols):
    valid_cols = base_cols.copy()
    for col in extra_cols:
        if col in df.columns:
            valid_cols.append(col)
    return valid_cols

# 4. Pipeline seguro
def create_pipeline():
    return Pipeline([
        ('preprocessor', ColumnTransformer([
            ('onehot', OneHotEncoder(handle_unknown='ignore'), ['Nombre Establecimiento'])
        ], remainder='passthrough')),
        ('model', HistGradientBoostingRegressor(
            max_iter=200,
            max_depth=5,
            early_stopping=True,
            random_state=42
        ))
    ])

# 5. Modelado por hospital con verificación robusta
resultados = {}
modelos = {}

for hospital in X['Nombre Establecimiento'].unique():
    mask = X['Nombre Establecimiento'] == hospital
    datos_hospital = X[mask].copy()
    y_hospital = y[mask]
    
    if len(datos_hospital) < 12:  # Mínimo 1 año de datos
        continue
        
    try:
        # Obtener columnas válidas para este hospital
        columnas_validas = get_valid_columns(datos_hospital, COLUMNAS_BASE, COLUMNAS_EXTRA)
        
        # Verificar que tenemos datos suficientes
        if len(columnas_validas) < 3:
            continue
            
        # Dividir en train-test temporal
        split_idx = int(0.8 * len(datos_hospital))
        X_train = datos_hospital.iloc[:split_idx][columnas_validas]
        X_test = datos_hospital.iloc[split_idx:][columnas_validas]
        y_train = y_hospital.iloc[:split_idx]
        y_test = y_hospital.iloc[split_idx:]
        
        # Entrenar modelo
        modelo = create_pipeline()
        modelo.fit(X_train, y_train)
        
        # Evaluar
        y_pred = modelo.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        
        # Guardar resultados
        resultados[hospital] = {
            'MAE': mae,
            'n_train': len(X_train),
            'n_test': len(X_test),
            'columnas_usadas': ', '.join(columnas_validas)
        }
        modelos[hospital] = modelo
        
    except Exception as e:
        print(f"Error con {hospital[:30]}...: {str(e)}")
        continue

# 6. Resultados consolidados
df_resultados = pd.DataFrame.from_dict(resultados, orient='index')
print("\nResumen de resultados:")
print(df_resultados.describe())

print("\nMejores hospitales por MAE:")
print(df_resultados.sort_values('MAE').head(10))

print("\nPeores hospitales por MAE:")
print(df_resultados.sort_values('MAE', ascending=False).head(10))


Resumen de resultados:
               MAE  n_train  n_test
count    52.000000     52.0    52.0
mean    164.795364    107.0    27.0
std     274.352609      0.0     0.0
min       4.882758    107.0    27.0
25%      45.422898    107.0    27.0
50%      73.471107    107.0    27.0
75%     173.620239    107.0    27.0
max    1749.854847    107.0    27.0

Mejores hospitales por MAE:
                                                          MAE  n_train  \
Hospital de Nancagua                                 4.882758      107   
Hospital de Mejillones                               4.927255      107   
Hospital de Lolol                                    5.778443      107   
Hospital de Pichidegua                              12.965703      107   
Hospital de Galvarino                               16.477268      107   
Hospital de Litueche                                18.039986      107   
Hospital Comunitario de Salud Familiar de El Ca...  22.788513      107   
Hospital Hanga Roa (Isla De Pas

In [95]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

# 1. Definir columnas base que SÍ existen en todos los hospitales
COLUMNAS_BASE = [
    'Nombre Establecimiento',
    'temperature_2m_max',
    'temperature_2m_min',
    'dias_desde_inicio'
]

# 2. Columnas adicionales (verificar existencia)
COLUMNAS_EXTRA = [
    'precipitation_sum',
    'winddirection_10m_dominant',
    'seno_mensual',
    'coseno_mensual',
    'ocupados_lag_1',
    'ocupados_lag_3',
    'ocupados_rolling_3'
]

# 3. Función para verificar columnas existentes
def get_valid_columns(df, base_cols, extra_cols):
    valid_cols = base_cols.copy()
    for col in extra_cols:
        if col in df.columns:
            valid_cols.append(col)
    return valid_cols

# 4. Pipeline seguro
def create_pipeline():
    return Pipeline([
        ('preprocessor', ColumnTransformer([
            ('onehot', OneHotEncoder(handle_unknown='ignore'), ['Nombre Establecimiento'])
        ], remainder='passthrough')),
        ('model', XGBRegressor(learning_rate=0.01,
            n_estimators=1000, 
            max_depth= 6,  
            min_child_weight= 3
        ))
    ])

# 5. Modelado por hospital con verificación robusta
resultados = {}
modelos = {}

for hospital in X['Nombre Establecimiento'].unique():
    mask = X['Nombre Establecimiento'] == hospital
    datos_hospital = X[mask].copy()
    y_hospital = y[mask]
    
    if len(datos_hospital) < 12:  # Mínimo 1 año de datos
        continue
        
    try:
        # Obtener columnas válidas para este hospital
        columnas_validas = get_valid_columns(datos_hospital, COLUMNAS_BASE, COLUMNAS_EXTRA)
        
        # Verificar que tenemos datos suficientes
        if len(columnas_validas) < 3:
            continue
            
        # Dividir en train-test temporal
        split_idx = int(0.8 * len(datos_hospital))
        X_train = datos_hospital.iloc[:split_idx][columnas_validas]
        X_test = datos_hospital.iloc[split_idx:][columnas_validas]
        y_train = y_hospital.iloc[:split_idx]
        y_test = y_hospital.iloc[split_idx:]
        
        # Entrenar modelo
        modelo = create_pipeline()
        modelo.fit(X_train, y_train)
        
        # Evaluar
        y_pred = modelo.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        
        # Guardar resultados
        resultados[hospital] = {
            'MAE': mae,
            'n_train': len(X_train),
            'n_test': len(X_test),
            'columnas_usadas': ', '.join(columnas_validas)
        }
        modelos[hospital] = modelo
        
    except Exception as e:
        print(f"Error con {hospital[:30]}...: {str(e)}")
        continue

# 6. Resultados consolidados
df_resultados = pd.DataFrame.from_dict(resultados, orient='index')
print("\nResumen de resultados:")
print(df_resultados.describe())

print("\nMejores hospitales por MAE:")
print(df_resultados.sort_values('MAE').head(10))

print("\nPeores hospitales por MAE:")
print(df_resultados.sort_values('MAE', ascending=False).head(10))


Resumen de resultados:
               MAE  n_train  n_test
count    52.000000     52.0    52.0
mean    151.103622    107.0    27.0
std     247.240704      0.0     0.0
min       4.029762    107.0    27.0
25%      37.455379    107.0    27.0
50%      85.634691    107.0    27.0
75%     144.675497    107.0    27.0
max    1566.173177    107.0    27.0

Mejores hospitales por MAE:
                                                          MAE  n_train  \
Hospital de Mejillones                               4.029762      107   
Hospital de Nancagua                                 4.385720      107   
Hospital Lord Cochrane                               4.796968      107   
Hospital de Lolol                                    4.899688      107   
Hospital de Corral                                   7.709790      107   
Hospital de Contulmo                                13.146884      107   
Hospital de Galvarino                               15.820166      107   
Hospital de Pichidegua         

In [86]:
from sklearn.model_selection import cross_val_score, TimeSeriesSplit

# Configurar CV para series temporales
tscv = TimeSeriesSplit(n_splits=5)

# Ejecutar validación cruzada
try:
    scores = cross_val_score(
        pipeline,
        X[feature_cols + ['Nombre Establecimiento']],  # Incluir columna categórica
        y,
        cv=tscv,
        scoring='neg_mean_absolute_error',
        error_score='raise'
    )
    print(f"MAE promedio: {-scores.mean():.2f}")
except Exception as e:
    print(f"Error: {str(e)}")

Error: A given column is not a column of the dataframe
